## Demos

### Load in the Database

In [ ]:
from astrodb_utils import load_astrodb
from simple import REFERENCE_TABLES

SCHEMA_PATH = "simple/schema.yaml" 
recreatedb = False #Set to True or False
db = load_astrodb(
     "SIMPLE.sqlite", 
     recreatedb=recreatedb,  
     reference_tables=REFERENCE_TABLES, 
     felis_schema=SCHEMA_PATH
     )

100%|██████████| 3619/3619 [00:20<00:00, 173.50it/s]


## Query the Database

### Using Astrodbkit Query Functions

##### What is Astrodbkit? (Link: https://astrodbkit.readthedocs.io/en/latest/#exploring-the-schema)
##### AstrodbKit is an astronomical database handler code built on top of SQLAlchemy. We can implement various functions from the python package to query the database in robust ways.

In [2]:
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u

#### Lets check the tables in the database

In [3]:
for table in db.metadata.tables:
    print(table)

CompanionList
CompanionParameters
Sources
Publications
CompanionRelationships
Gravities
Regimes
Instruments
Telescopes
ModeledParameters
Parameters
Names
Parallaxes
Photometry
PhotometryFilters
ProperMotions
RadialVelocities
RotationalParameters
Spectra
SpectralTypes
Versions


#### Now that we know the tables,lets do a more detailed search into the tables and their content

#### Lets look at the first 10 sources available within SIMPLE

In [4]:
db.query(db.Sources).limit(10).table()

source,ra,dec,epoch,equinox,shortname,reference,other_references,comments
str24,float64,float64,object,object,object,str11,object,object
2MASS J03552014+1439297,58.83375,14.658056,None,None,None,Cruz07,None,None
1RXS J115928.5-524717,179.864,-52.7886,None,2000,None,Hamb04.265,None,None
ULAS J114925.58-014343.2,177.3569,-1.7287,None,2000,None,Burn10.1885,None,None
DENIS-P J1756561-480509,269.2342,-48.086,None,2000,None,Phan08,None,None
PSO J024.4369+09.1360,24.4372,9.1366,None,2000,None,Skrz16,None,Best20 lists discovery as Skrz16; Best20a
SDSS J161420.50+004643.6,243.585,0.7785,None,2000,None,Hawl02,None,None
SDSS J094903.15+264944.2,147.2633,26.8288,None,2000,None,Schm10.1808,None,None
SDSS J083717.21-000018.0,129.3215835,-0.004972,None,None,0837-0000,Legg00,None,None
2MASS J06080232-2944590,92.0097,-29.7497,None,2000,None,Cruz03,None,None


#### Thats a lot of sources! Lets say you just want to find all sources that start with "WISE" 
#### We can use the search_object function for this
#### Search_Object documentation: https://astrodbkit.readthedocs.io/en/latest/#identifier-name-search

In [5]:
db.search_object('wise', fmt='astropy')

Using table 'Sources' with columns ['source'] for matching object names
Using table 'Names' with columns ['other_name'] for matching object names


source,ra,dec,epoch,equinox,shortname,reference,other_references,comments
str32,float64,float64,object,object,object,str11,object,object
1RXS J115928.5-524717,179.864,-52.7886,None,2000,None,Hamb04.265,None,None
2MASS J00011217+1535355,0.300708,15.593194,None,None,0001+1535,Knap04,None,None
2MASS J00034227-2822410,0.926125,-28.378056,None,None,0003-2822,Cruz07,None,None
2MASS J00040288-6410358,1.012,-64.176611,None,None,0004-6410,Kirk10,None,None
2MASS J00043484-4044058,1.145166,-40.734944,None,None,0004-4044BC,Eros99,None,None
2MASS J00044144-2058298,1.1728,-20.975,None,2000,None,Kend07,None,None
2MASS J00045753-1709369,1.2398,-17.1603,None,2000,None,Crif05,None,None
2MASS J00054844-2157196,1.4519,-21.9555,None,2000,None,Reyl04,None,None
2MASS J00070787-2458042,1.7827917,-24.967833,None,None,0007-2458,Reyl04,None,None


#### So you found the object youre looking for - lets now narrow down that query.

#### Lets see many names are associated with the object WISEPC J0333

In [6]:
db.search_object('WISEPC J0333', resolve_simbad=True, output_table='Names')

No Simbad match for WISEPC J0333
Including Simbad names, searching for: ['WISEPC J0333']
Using table 'Sources' with columns ['source'] for matching object names
Using table 'Names' with columns ['other_name'] for matching object names


source,other_name
str26,str26
WISEPC J033349.34-585618.7,CWISE J033349.31-585619.8
WISEPC J033349.34-585618.7,VHS J033349.30-585620.2
WISEPC J033349.34-585618.7,WISEPC J033349.34-585618.7


#### We know the source exist, so now we want to see all the data associated with the source. 
#### Inventory documentation: https://astrodbkit.readthedocs.io/en/latest/#inventory-search

In [7]:
data = db.inventory('WISEPC J033349.34-585618.7', pretty_print=True)

{
    "Sources": [
        {
            "source": "WISEPC J033349.34-585618.7",
            "ra": 53.4557,
            "dec": -58.9384,
            "epoch": null,
            "equinox": "2000",
            "shortname": null,
            "reference": "Kirk11",
            "other_references": null,
            "comments": null
        }
    ],
    "Names": [
        {
            "other_name": "CWISE J033349.31-585619.8"
        },
        {
            "other_name": "VHS J033349.30-585620.2"
        },
        {
            "other_name": "WISEPC J033349.34-585618.7"
        }
    ],
    "Parallaxes": [
        {
            "parallax": 46.2,
            "parallax_error": 3.7,
            "adopted": true,
            "comments": null,
            "reference": "Kirk21"
        }
    ],
    "Photometry": [
        {
            "band": "IRAC.I1",
            "magnitude": 13.59,
            "magnitude_error": 0.02,
            "telescope": "Spitzer",
            "epoch": null,
            

#### Now lets do a broader search, say you want to find all objects with the publication reference "Cruz18" 

##### Search string documentation: https://astrodbkit.readthedocs.io/en/latest/#full-string-search

#### Within astrodbkit, we allow for the parameter format (fmt) so that you can decide how you wish to visualize your query. Your options are pandas, astropy, or default (default is an astropy table)

In [ ]:
db.search_string('Cruz18', fuzzy_search=False, fmt='astropy')  #fmt options: pandas, astropy/table, default

Sources
         source               ra      ... other_references   comments  
----------------------- ------------- ... ---------------- ------------
2MASS J15382417-1953116     234.60071 ...             None         None
2MASS J00110940+5149236      2.789182 ...             None         None
2MASS J15063706+2759544     226.65421 ...             None       galaxy
2MASS J05583706-1339123     89.654445 ...             None         None
2MASS J23490528+1833150       357.272 ...             None         None
2MASS J23202927+4123415     350.12167 ...             None         None
2MASS J22433237-1525260    340.884859 ...             None         None
2MASS J12490458-3454080     192.26979 ...             None       galaxy
2MASS J23364145+4822480    354.172713 ...             None         None
2MASS J05574102-1333264     89.420954 ...             None        not M
                    ...           ... ...              ...          ...
2MASS J05574229-1333156     89.426209 ...             No

{'Sources': <Table length=24>
          source               ra      ... other_references   comments  
          str23             float64    ...      object         object   
 ----------------------- ------------- ... ---------------- ------------
 2MASS J15382417-1953116     234.60071 ...             None         None
 2MASS J00110940+5149236      2.789182 ...             None         None
 2MASS J15063706+2759544     226.65421 ...             None       galaxy
 2MASS J05583706-1339123     89.654445 ...             None         None
 2MASS J23490528+1833150       357.272 ...             None         None
 2MASS J23202927+4123415     350.12167 ...             None         None
 2MASS J22433237-1525260    340.884859 ...             None         None
 2MASS J12490458-3454080     192.26979 ...             None       galaxy
 2MASS J23364145+4822480    354.172713 ...             None         None
                     ...           ... ...              ...          ...
 2MASS J05574229-1333

#### Next example is query region, where you can use SkyCoord to create a coordinates object that query_region uses to query the database, in this example the Sources table. Make sure to use as accurate as a coordinate as possible. 
#### Query_Region Documentation: https://astrodbkit.readthedocs.io/en/latest/#region-spatial-search

##### Notes on Query Region: 
##### If you have the exact coordinates of the object you are searching you can use query_region as shown. However, if you don't you can enter the optional parameter "radius=x" to allow for a larger search radius, otherwise you will get back an empty query if your coordinates aren't precise enough

In [ ]:
# An example of query with precise coordinates ----------
precise_coords = db.query_region(SkyCoord(144.395292, 29.528028, frame='icrs', unit='deg'), output_table='Sources', ra_col='ra', dec_col='dec')  
print(f'Table Results: \n {precise_coords}\n')

# An example that queries the database incorrectly
imprecise_coords = db.query_region(SkyCoord(144.3, 29.5, frame='icrs', unit='deg'), output_table='Sources', ra_col='ra', dec_col='dec') 
print(f'Table Results: \n {imprecise_coords}\n')

# An example that queries the database using the radius parameter
larger_search_area = db.query_region(SkyCoord(144.395, 29.528, radius=0.5*u.deg, frame='icrs', unit='deg'), output_table='Sources', ra_col='ra', dec_col='dec',)
print(f'Table Results: \n {len(larger_search_area)}\n')

Query results with approximate coordinates (144.3, 29.5): 0
Table Results: 
 <No columns>
The query returned an empty result because the coordinates aren't precise enough.
Try using more precise coordinates or increasing the search radius.

Query results with precise coordinates (144.395292, 29.528028): 1
Table Results: 
          source             ra        dec    ... other_references comments
----------------------- ---------- --------- ... ---------------- --------
2MASS J09373487+2931409 144.395292 29.528028 ...             None     None


#### Have a more complex query? Let use astrodb_utils.sql_query to run a SQL query directly 
##### AstrodbUtils's function sql_query acts as a wrapped for direct SQL queries, taking in two parameters: query (str), and fmt (str: pandas, astropy, defualt). 

In [32]:
from sqlalchemy import and_, or_

#Uncomment the following lines to see how to use AND and OR in queries

# Query with multiple conditions
print("\nQuery with multiple conditions, filtering sources with dec > 0 and ra < 200 degrees:")
print(f"Showing the first 10 results out of {len(db.query(db.Sources).filter(db.Sources.c.dec > 0).filter(db.Sources.c.ra < 200).all())} sources:")
db.query(db.Sources).filter(db.Sources.c.dec > 0).filter(db.Sources.c.ra < 200).limit(10).table()

# #  Query with AND
# print("\nQuery with AND, filtering sources with dec > 0 and ra > 200 degrees:")
# print(f"Showing the first 10 results out of {len(db.query(db.Sources).filter(and_(db.Sources.c.dec > 0, db.Sources.c.ra > 200)).all())} sources:")
# db.query(db.Sources).filter(and_(db.Sources.c.dec > 0, db.Sources.c.ra > 200)).limit(10).table()


# #  Query with OR
# print("\nQuery with OR, filtering sources with dec < 0 or ra > 200 degrees:")
# print(f"Showing the first 10 results out of {len(db.query(db.Sources).filter(or_(db.Sources.c.dec < 0, db.Sources.c.ra > 200)).all())} sources:")
# db.query(db.Sources).filter(or_(db.Sources.c.dec < 0, db.Sources.c.ra > 200)).limit(10).table()



Query with multiple conditions, filtering sources with dec > 0 and ra < 200 degrees:
Showing the first 10 results out of 1334 sources:


source,ra,dec,epoch,equinox,shortname,reference,other_references,comments
str24,float64,float64,object,object,object,str11,object,object
2MASS J03552014+1439297,58.83375,14.658056,None,None,None,Cruz07,None,None
PSO J024.4369+09.1360,24.4372,9.1366,None,2000,None,Skrz16,None,Best20 lists discovery as Skrz16; Best20a
SDSS J094903.15+264944.2,147.2633,26.8288,None,2000,None,Schm10.1808,None,None
2MASS J09373487+2931409,144.395292,29.528028,None,None,0937+2931,Burg02.421,None,None
HIP 6407B,20.5707,3.5232,None,2000,None,Deac14.119,None,None
2MASS J11260310+4819256,171.5128,48.3239,None,2000,None,Kell15,None,None
LHS 1937,115.27833,17.645833,None,None,None,Luyt79.5,None,None
ULAS J095047.28+011734.3,147.6971,1.2925,None,2000,None,Burn13,None,None
2MASSI J1029216+162652,157.3404,16.4477,None,2000,None,Kirk00,None,None


#### Query the count of sources in the database

In [33]:
# Query the total number of objects within SIMPLE
result = db.sql_query("SELECT COUNT(*) FROM Sources", fmt='astropy')
total_objects = result[0][0]


print(f"Total number of objects in the SIMPLE database: {total_objects}")

Total number of objects in the SIMPLE database: 3619


#### Query Sources by String 

In [34]:
wise_sources_query = "SELECT * FROM Sources WHERE source LIKE '%WISE%'"
wise_sources = db.sql_query(wise_sources_query, fmt='astropy')

print(f"Found {len(wise_sources)} WISE sources in the database")
print("\nFirst Three WISE Sources:")
for source in wise_sources[:3]:  # Display first 3 sources
    print(source)

Found 616 WISE sources in the database

First Three WISE Sources:
          source                   ra               dec         epoch equinox shortname reference other_references comments
-------------------------- ----------------- ------------------ ----- ------- --------- --------- ---------------- --------
CWISEP J230158.30-645858.3 345.4929166666666 -64.98286111111112  None    None      None Meis20.74             None     None
         source             ra     dec    epoch equinox shortname reference other_references comments
------------------------ ------- -------- ----- ------- --------- --------- ---------------- --------
WISE J200403.17-263751.7 301.013 -26.6293  None    2000      None    Thom13             None     None
          source                  ra                 dec         epoch equinox shortname reference  other_references comments
------------------------- ------------------ ------------------- ----- ------- --------- ---------- ---------------- --------
WISE

#### Query by Program ID ("PID")

In [37]:
spectra_pid_query = "SELECT * FROM Spectra WHERE reference LIKE 'PID%'"
print(f"Found {len(db.sql_query(spectra_pid_query))} spectra with PID references in the database")
db.sql_query(spectra_pid_query, fmt='astropy')


Found 55 spectra with PID references in the database


source,access_url,original_spectrum,local_spectrum,regime,telescope,instrument,mode,observation_date,comments,reference,other_references
str24,str67,object,str43,str3,str7,str3,str7,str26,object,str8,object
VB 10,https://s3.amazonaws.com/bdnyc/Gl+752B+%285-35um%29.txt,None,$BDNYC_spectra/IRS/Gl 752B (5-35um).txt,mir,Spitzer,IRS,Missing,2005-10-11 00:00:00.000000,None,PID29,None
2MASS J10042066+5022596,https://s3.amazonaws.com/bdnyc/IRS/G196-3B.txt,None,$BDNYC_spectra/IRS/G196-3B.txt,mir,Spitzer,IRS,Missing,2004-04-17 04:40:11.761000,None,PID51,None
2MASS J10224821+5825453,https://s3.amazonaws.com/bdnyc/IRS/1022%2B5825.txt,None,$BDNYC_spectra/IRS/1022+5825.txt,mir,Spitzer,IRS,Missing,2005-04-16 13:45:42.752000,None,PID3136,None
2MASS J12392727+5515371,https://s3.amazonaws.com/bdnyc/1239%2B5515.txt,None,$BDNYC_spectra/IRS/1239+5515.txt,mir,Spitzer,IRS,Missing,2004-04-18 02:01:06.605000,None,PID51,None
2MASS J00001354+2554180,https://s3.amazonaws.com/bdnyc/IRS/0000%2B2554%20%285-14.5um%29.txt,None,$BDNYC_spectra/IRS/0000+2554 (5-14.5um).txt,mir,Spitzer,IRS,Missing,2005-12-18 00:00:00.000000,None,PID20514,None
2MASS J07464256+2000321,https://s3.amazonaws.com/bdnyc/0746%2B2000.txt,None,$BDNYC_spectra/IRS/0746+2000.txt,mir,Spitzer,IRS,Missing,2004-03-27 00:32:11.624000,None,PID51,None
2MASS J20575409-0252302,https://s3.amazonaws.com/bdnyc/2057-0252.txt,None,$BDNYC_spectra/IRS/2057-0252.txt,mir,Spitzer,IRS,SL,2005-05-25 01:56:38.317000,None,PID3136,None
LP 440-52,https://s3.amazonaws.com/bdnyc/IRS/LHS377.txt,None,$BDNYC_spectra/IRS/LHS377.txt,mir,Spitzer,IRS,Missing,2005-07-01 00:00:00.000000,None,PID251,None
2MASS J12373919+6526148,https://s3.amazonaws.com/bdnyc/IRS/1237%2B6526.txt,None,$BDNYC_spectra/IRS/1237+6526.txt,mir,Spitzer,IRS,SL,2004-03-03 20:10:57.708000,None,PID51,None


### Using SQLAlchemy Query Functions

#### SQL via SQLalchemy 

In [16]:
from sqlalchemy import text

# Using connection from the existing database
with db.engine.connect() as conn:
    
    # Query all sources
    result = conn.execute(text("SELECT * FROM Sources LIMIT 10"))
    print("First 10 Sources (Raw SQL):")
    for row in result:
        print(f"Source: {row.source}, RA: {row.ra}, Dec: {row.dec}")

    # Find spectra with a specific instrument
    result = conn.execute(
        text("SELECT * FROM Spectra WHERE instrument = :instrument LIMIT 5"),
        {"instrument": "IRS"}
    )
    print("\nSpectra from IRS instrument (first 5):")
    for row in result:
        print(f"Source: {row.source}, Telescope: {row.telescope}")

        # Join query - find all spectra for sources with names containing "WISE"
        result = conn.execute(
            text("""
            SELECT Sources.source, Spectra.telescope, Spectra.instrument
            FROM Sources 
            JOIN Spectra ON Sources.source = Spectra.source
            WHERE Sources.source LIKE :pattern
            LIMIT 5
            """),
            {"pattern": "%WISE%"}
        )
        print("\nWISE sources with spectra (first 5):")
        for row in result:
            print(f"Source: {row.source}, Telescope: {row.telescope}, Instrument: {row.instrument}")
            
        # Aggregate query - count sources by publication reference
        result = conn.execute(
            text("""
            SELECT reference, COUNT(*) as count 
            FROM Sources 
            GROUP BY reference 
            ORDER BY count DESC
            LIMIT 3
            """)
        )
        print("\nTop 3 references by number of sources:")
        for row in result:
            print(f"Reference: {row.reference}, Count: {row.count}")


First 10 Sources (Raw SQL):
Source: 2MASS J03552014+1439297, RA: 58.83375, Dec: 14.658056
Source: 1RXS J115928.5-524717, RA: 179.864, Dec: -52.7886
Source: ULAS J114925.58-014343.2, RA: 177.3569, Dec: -1.7287
Source: DENIS-P J1756561-480509, RA: 269.2342, Dec: -48.086
Source: PSO J024.4369+09.1360, RA: 24.4372, Dec: 9.1366
Source: SDSS J161420.50+004643.6, RA: 243.585, Dec: 0.7785
Source: SDSS J094903.15+264944.2, RA: 147.2633, Dec: 26.8288
Source: SDSS J083717.21-000018.0, RA: 129.3215835, Dec: -0.004972
Source: 2MASS J06080232-2944590, RA: 92.0097, Dec: -29.7497
Source: 2MASS J09373487+2931409, RA: 144.395292, Dec: 29.528028

Spectra from IRS instrument (first 5):
Source: SDSS J083717.21-000018.0, Telescope: Spitzer

WISE sources with spectra (first 5):
Source: WISE J235402.77+024015.0, Telescope: JWST, Instrument: MIRI
Source: WISE J235402.77+024015.0, Telescope: JWST, Instrument: NIRSpec
Source: WISE J235402.77+024015.0, Telescope: HST, Instrument: WFC3
Source: WISEP J031325.96+780